In [1]:
%%capture --no-stderr
!pip install composio_langchain==0.8.0 langgraph langchain_openai

In [6]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["COMPOSIO_API_KEY"] = userdata.get('COMPOSIO_API_KEY')

In [7]:
from typing import Literal
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode


# composio = Composio()
# user_id = "pg-test-f211b19e-0250-449c-a"  # Use your email or a unique identifier


In [8]:
from composio import Composio
from composio_langchain import LangchainProvider

composio = Composio(provider=LangchainProvider())

In [9]:
# Get the tools (replace user_id etc as appropriate)
tools = composio.tools.get(
    user_id="pg-test-f211b19e-0250-449c-a21c-166f2ece1281",
    tools=[
        "GOOGLECALENDAR_FIND_FREE_SLOTS",
        "GOOGLECALENDAR_CREATE_EVENT",
        "GMAIL_CREATE_EMAIL_DRAFT"
    ]
)

In [15]:
tools

[StructuredTool(name='GMAIL_CREATE_EMAIL_DRAFT', description='Creates a Gmail email draft, requiring at least one of recipient_email, cc, or bcc must be provided. Atleast one of subject or body must be provided. Supports To/Cc/Bcc, subject, plain/HTML body (ensure `is_html=True` for HTML), attachments, and threading.', args_schema=<class 'composio.utils.shared.CreateEmailDraftRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function LangchainProvider._wrap_action.<locals>.function at 0x791c1d52fe20>),
 StructuredTool(name='GOOGLECALENDAR_CREATE_EVENT', description='Create a Google Calendar event using `start_datetime` plus duration fields to derive the end time. Requires calendar write access. The organizer is added as an attendee unless `exclude_organizer` is True. Example request to create event for 1 hour 30 minutes: { "calendar_id": "primary", "start_datetime": "2025-01-16T13:00:00", "timezone": "America/New_York", "event_duration_hour": 1, "event_duration_min

In [10]:
tool_node = ToolNode(tools)

In [11]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [12]:
model_with_tools = model.bind_tools(tools)

In [53]:
model_with_tools.kwargs

{'tools': [{'type': 'function',
   'function': {'name': 'GMAIL_CREATE_EMAIL_DRAFT',
    'description': 'Creates a Gmail email draft, requiring at least one of recipient_email, cc, or bcc must be provided. Atleast one of subject or body must be provided. Supports To/Cc/Bcc, subject, plain/HTML body (ensure `is_html=True` for HTML), attachments, and threading.',
    'parameters': {'properties': {'attachment': {'default': None,
       'description': 'File to attach to the email.',
       'type': 'string'},
      'bcc': {'default': [],
       'description': "Blind Carbon Copy (BCC) recipients' email addresses. Atleast one of cc, bcc, or recipient_email must be provided.",
       'items': {'type': 'string'},
       'type': 'array'},
      'body': {'default': None,
       'description': 'Email body content (plain text or HTML); `is_html` must be True if HTML. Either subject or body must be provided. Please provide a value of type string.',
       'type': 'string'},
      'cc': {'default': []

In [13]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

In [14]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [15]:
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# Step 9: Compile workflow
app = workflow.compile()

In [16]:
for chunk in app.stream(
    {
        "messages": [
            ("human",
             "Find a 30-minute free slot tomorrow afternoon, create a Google Meet event called 'Project Sync with John Doe', and draft a Gmail invitation to John at johndoe@example.com.")
        ]
    },
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

Find a 30-minute free slot tomorrow afternoon, create a Google Meet event called 'Project Sync with John Doe', and draft a Gmail invitation to John at johndoe@example.com.
================================== Ai Message ==================================
Tool Calls:
  GOOGLECALENDAR_FIND_FREE_SLOTS (call_5WuGiKHla89CQIpQFARcq5a5)
 Call ID: call_5WuGiKHla89CQIpQFARcq5a5
  Args:
    time_min: 2023-11-30T12:00:00
    time_max: 2023-11-30T17:00:00
    timezone: UTC
================================= Tool Message =================================
Name: GOOGLECALENDAR_FIND_FREE_SLOTS

{"data": {"calendars": {"primary": {"busy": [], "free": [{"end": "2023-11-30T17:00:00+00:00", "start": "2023-11-30T12:00:00+00:00"}]}}, "kind": "calendar#freeBusy", "timeMax": "2023-11-30T17:00:00.000Z", "timeMin": "2023-11-30T12:00:00.000Z"}, "error": null, "successful": true}
================================== Ai Message ==========